# Network Creation

Let's create a small Neural Network with 3 hidden layers for testing

In [1]:
import numpy as np
import pandas as pd
from NeuralNetwork import *
from utils import *

In [2]:
col_names = ('Input1', 'Input2', 'Input3', 'Input4', 'Input5', 'Input6',
             'Input7', 'Input8', 'Input9', 'Input10', 'TARGET_x', 'TARGET_y', 'TARGET_z')
data = pd.read_csv("../ML-23-PRJ-Package/ML-CUP23-TR.csv",
                   skiprows=7, usecols=[i for i in range(1, 14)], names=col_names)

data = data.sample(frac=1)
# SPLIT 80/20 FOR TRAIN/TEST
folds = np.array_split(data, 5)
tr_data = folds[1:]
test_data = folds[0]

epochs = 300
params = {
    "eta": 0.00002,
    "mb": 1,
    "momentum": 0.99,
    "epochs": epochs,
    "clip_value": None,
    "hid_act_fun": "tanh",
    "out_act_fun": "tanh",
    "cost_fun": "mse",
    "ridge_lambda": 0.0000005,
    "lasso_lambda": None,
    "decay_max_steps": None,
    "decay_min_value": None
}

parallel_grid_search(5, tr_data)

In [4]:
col_names = ('Input1', 'Input2', 'Input3', 'Input4', 'Input5', 'Input6',
             'Input7', 'Input8', 'Input9', 'Input10', 'TARGET_x', 'TARGET_y', 'TARGET_z')
data = pd.read_csv("../ML-23-PRJ-Package/ML-CUP23-TR.csv",
                   skiprows=7, usecols=[i for i in range(1, 14)], names=col_names)

data = data.sample(frac=1)
#SPLIT 80/20 FOR TRAIN/TEST
folds = np.array_split(data, 5)
tr_data = folds[1:]
test_data = folds[0]

epochs=300
params={
    "eta" : 0.00002,
    "mb" : 1,
    "momentum" : 0.99,
    "epochs" : epochs,
    "clip_value" : None,
    "hid_act_fun" : "tanh",
    "out_act_fun" : "tanh",
    "cost_fun" : "mse",
    "ridge_lambda":0.0000005,
    "lasso_lambda": None,
    "decay_max_steps": None,
    "decay_min_value": None
}

n=20
test_errors=np.zeros((n,epochs))
training_errors=np.zeros((n,epochs))
acc_test_errors=np.zeros((n,epochs))
acc_train_errors=np.zeros((n,epochs))
for i in range(n):
    net = NeuralNetwork()

    n_inputs = tr_data.shape[1] - 2

    net.add_input_layer(n_inputs)

    net.add_hidden_layer(n_inputs, 50)

    net.add_output_layer(50, 3)
    test_error,training_error,acc_test,acc_train=net.train(tr_data,params,test_data=test_data)
    test_errors[i]=test_error
    training_errors[i]=training_error
    acc_test_errors[i]=acc_test
    acc_train_errors[i]=acc_train

print(f"MSE: mean = {np.mean(test_errors[:,-1])}, Var = {np.var(test_errors[:,-1])}")
#print(f"Accuracy: mean = {np.mean(acc_test_errors[:,-1])}, std = {np.std(acc_test_errors[:,-1])}")


/home/nsiva/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


AttributeError: 'list' object has no attribute 'shape'